In [1]:
import os
import rasterio
from shapely import geometry
from rasterio.mask import mask

In [2]:
# all_sites = ['herschel', 'herschel_val', 'horton', 'horton_val', 'kolguev', 'kolguev_val', 'lena', 'lena_val', 'unvalidated']
all_sites = ['horton']
data_pathnames = []
for site in all_sites:
    for file in os.listdir('../all_data/' + site):
        if file.endswith(".tiff"):
            data_pathnames.append(os.path.join('../all_data/' + site, file))
            
print(data_pathnames[0][11:-5])

/horton/Horton_02_2019-09-07_datacube


In [3]:
# Takes a Rasterio dataset and splits it into squares of dimensions squareDim * squareDim
def splitImageIntoCells(img, filename, squareDim):
    numberOfCellsWide = img.shape[1] // squareDim
    numberOfCellsHigh = img.shape[0] // squareDim
    x, y = 0, 0
    count = 0
    for hc in range(numberOfCellsHigh):
        y = hc * squareDim
        for wc in range(numberOfCellsWide):
            x = wc * squareDim
            geom = getTileGeom(img.transform, x, y, squareDim)
            getCellFromGeom(img, geom, filename, count)
            count = count + 1

# Generate a bounding box from the pixel-wise coordinates using the original datasets transform property
def getTileGeom(transform, x, y, squareDim):
    corner1 = (x, y) * transform
    corner2 = (x + squareDim, y + squareDim) * transform
    return geometry.box(corner1[0], corner1[1],
                        corner2[0], corner2[1])

# Crop the dataset using the generated box and write it out as a GeoTIFF
def getCellFromGeom(img, geom, filename, count):
    crop, cropTransform = mask(img, [geom], crop=True)
    writeImageAsGeoTIFF(crop,
                        cropTransform,
                        img.meta,
                        img.crs,
                        filename+"_"+str(count))

# Write the passed in dataset as a GeoTIFF
def writeImageAsGeoTIFF(img, transform, metadata, crs, filename):
    metadata.update({"driver":"GTiff",
                     "height":img.shape[1],
                     "width":img.shape[2],
                     "transform": transform,
                     "crs": crs})
    with rasterio.open(filename+".tif", "w", **metadata) as dest:
        dest.write(img)

In [4]:
for path in data_pathnames:
    with rasterio.open(path, 'r') as src:
        length = src.shape[0]
        file_name = '../test' + path[11:-5]
        splitImageIntoCells(src, file_name, int(length/2))

/tmp/ipykernel_811801/2743907270.py:17: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
/tmp/ipykernel_811801/2743907270.py:18: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim, y + squareDim) * transform
/tmp/ipykernel_811801/2743907270.py:17: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
/tmp/ipykernel_811801/2743907270.py:18: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim, y + squareDim) * transform
/tmp/ipykernel_811801/2743907270.py:17: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner1 = (x, y) * transform
/tmp/ipykernel_811801/2743907270.py:18: DeprecationWarning: Right multiplication will be prohibited in version 3.0
  corner2 = (x + squareDim, y + squareDim) * transform
/tmp/ipykernel_811801/2743907270.py:17: DeprecationW